# Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.parallel
import torch.utils.data

# Load data

In [2]:
path_relative = 'data_sources'
df_ratings =pd.read_csv(path_relative + '/ml-latest-small/ratings.csv')
df_ratings = df_ratings[['userId', 'movieId', 'rating']]
df_ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


## Pivot table

In [3]:
df_input = pd.pivot_table(df_ratings, values='rating', index=['userId'], columns=['movieId'], aggfunc=np.sum)
df_input.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Deal with NaN

In [4]:
df_input = df_input.fillna(-1)
df_input.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,-1.0,4.0,-1.0,-1.0,4.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


# Train/Test split

In [5]:
X = df_input.sample(frac=1).round() #Round ratings
len_train = int(np.round(0.8*len(X)))
X_train = X.head(len_train)
X_test = X.tail(len_train)

## Converting to PyTorch tensors

In [6]:
training_set = X_train.values
test_set = X_test.values

In [7]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [8]:
print(training_set)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 5., -1., -1.,  ..., -1., -1., -1.],
        [ 4.,  4., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])


In [9]:
print(test_set)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [ 2.,  0., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])


## Convert the ratings into binary ratings 1 (Liked) or 0 (Did not liked)

In [10]:
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1
print(test_set)
print(training_set)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [ 0.,  0., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])
tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [ 1.,  1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])


# Building the RBM

In [11]:
class RBM():
    def __init__(self, nv, nh, batch_size, nb_epoch, k_steps, learning_rate, verbose):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
        self.nh = nh
        self.nv = nv
        self.verbose = verbose
        self.batch_size = batch_size
        self.nb_epoch = nb_epoch
        self.k_steps = k_steps
        self.learning_rate = learning_rate
        
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y):
        wy = torch.mm(y, self.W) # ojo con esto
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def update_weights(self, v0, vk, ph0, phk):
        learning_rate = self.learning_rate
        self.W += learning_rate*(torch.t(torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)))
        self.b += learning_rate*(torch.sum((v0 - vk), 0))
        self.a += learning_rate*(torch.sum((ph0 - phk), 0))
        
    def train(self, training_set):
        batch_size = self.batch_size
        nb_epoch = self.nb_epoch
        k_steps = self.k_steps
        verbose = self.verbose
        
        #Training the RBM
        
        for epoch in range(1, nb_epoch, + 1):
            train_loss = 0
            s = 0.
            nb_users = len(training_set)
            for id_user in range(0, nb_users - batch_size, batch_size):
                vk = training_set[id_user: id_user+batch_size]
                v0 = training_set[id_user: id_user+batch_size]
                ph0,_ = self.sample_h(v0)
                for k in range(k_steps):
                    _,hk = self.sample_h(vk)
                    _,vk = self.sample_v(hk)
                    vk[v0<0] = v0[v0<0]
                phk,_ = self.sample_h(vk)
                self.update_weights(v0, vk, ph0, phk)
                train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
                s += 1.
            if verbose:
                print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
                
    
    def evaluate(self, test_set):
        verbose = self.verbose
        nb_users = len(test_set)
        
        #testing the RBM
        test_loss = 0
        s = 0.
        
        for id_user in range(nb_users):
            v = training_set[id_user: id_user+1]
            vt = test_set[id_user: id_user+1]
            if len(vt[vt>=0]) > 0:
                _,h = self.sample_h(v)
                _,v = self.sample_v(h)
                test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
                s+= 1.
        if verbose:
            print('test loss: '+str(test_loss/s))
        return test_loss/s
    
    def predict(self, v_user):
        _,h = self.sample_h(v_user)
        _,v = self.sample_v(h)
        return v

## Hyperparameters

In [12]:
nv = len(training_set[0])
nh = 100
batch_size = 100
nb_epoch = 10
k_steps = 10
learning_rate = 1
verbose = True

# Train model

In [13]:
rbm = RBM(nv, nh, batch_size, nb_epoch, k_steps, learning_rate, verbose)
rbm.train(training_set)

epoch: 1 loss: tensor(0.4586)
epoch: 2 loss: tensor(0.3094)
epoch: 3 loss: tensor(0.2692)
epoch: 4 loss: tensor(0.2540)
epoch: 5 loss: tensor(0.2399)
epoch: 6 loss: tensor(0.2339)
epoch: 7 loss: tensor(0.2336)
epoch: 8 loss: tensor(0.2294)
epoch: 9 loss: tensor(0.2284)


## Evaluation

In [14]:
rbm.evaluate(test_set)

test loss: tensor(0.2407)


tensor(0.2407)

### Obtain an individual prediction

In [15]:
v_test = test_set[0:1]
v_pred = rbm.predict(v_test)

#### Check movie for that prediction

In [16]:
id_movies = ([v_pred==1][0][0]) & ([v_test==-1][0][0])
id_movies = id_movies.tolist()

#### Combine with movie titles and see results for units with missing values

In [17]:
df_recom = pd.DataFrame({'id_check': id_movies})
df_movies = pd.read_csv(path_relative  + '/ml-latest-small/movies.csv')
df_recom = df_recom.reset_index().rename(columns={'index':'movieId'}).merge(df_movies)

print('Recommended movies: ')
print(df_recom[df_recom['id_check']==True]['title'].head(10))

Recommended movies: 
0                       Toy Story (1995)
1                         Jumanji (1995)
2                Grumpier Old Men (1995)
6                         Sabrina (1995)
7                    Tom and Huck (1995)
10        American President, The (1995)
11    Dracula: Dead and Loving It (1995)
12                          Balto (1995)
15                         Casino (1995)
16          Sense and Sensibility (1995)
Name: title, dtype: object


# Building the RBM
## (_Using tensorflow_)

### Class that defines the behavior of the RBM

In [18]:
import tensorflow as tf
class RBM_tensorflow(object):
    
    def __init__(self, input_size, output_size, lr=1.0, batchsize=100):
        """
        m: Number of neurons in visible layer
        n: Number of neurons in hidden layer
        """
        
        #Defining the hyperparameters
        self._input_size = input_size #Size of Visible
        self._output_size = output_size #Size of output
        self.learning_rate = lr #The step used in gradient descient
        self.batchsize = batchsize #The size of how much data will be used for training per sub iteration
        
        #Initializing weights and biases as matrices full of zeroes
        self.w = tf.zeros([input_size, output_size], np.float32) #Creates and initializes the weights with 0
        self.hb = tf.zeros([output_size], np.float32) #Creates and initializes the hidden biases with 0
        self.vb = tf.zeros([input_size], np.float32) #Creates and initalizes the visible biases with 0
        
    #Forward Pass
    def prob_h_given_v(self, visible, w, hb):
        #Sigmoid
        return tf.nn.sigmoid(tf.matmul(visible, w) + hb)
    
    #Backward Pass
    def prob_v_given_h(self, hidden, w, vb):
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w)) + vb)
    
    #Generate the sample probability
    def sample_prob(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))
    
    #Training method for the model
    def train(self, X, epochs=10):
        loss=[]
        for epoch in range(epochs):
            
            #For each step/batch
            for start, end in zip(range(0, len(X), self.batchsize), range(self.batchsize, len(X), self.batchsize)):
                batch = X[start:end]
                
                #Initialize with sample probabilities
                h0 = self.sample_prob(self.prob_h_given_v(batch, self.w, self.hb))
                v1 = self.sample_prob(self.prob_v_given_h(h0, self.w, self.vb))
                h1 = self.prob_h_given_v(v1, self.w, self.hb)
                
                #Create the Gradients
                positive_grad = tf.matmul(tf.transpose(batch), h0)
                negative_grad = tf.matmul(tf.transpose(v1), h1)
                
                #Update learning rates
                self.w = self.w + self.learning_rate * (positive_grad - negative_grad) / tf.dtypes.cast(tf.shape(batch)[0], tf.float32)
                self.vb = self.vb + self.learning_rate * tf.reduce_mean(batch - v1, 0)
                self.hb = self.hb + self.learning_rate * tf.reduce_mean(h0 - h1, 0)
            
            #Find the error rate
            err = tf.reduce_mean(tf.square(batch - v1))
            print ('Epoch: %d' % epoch, 'reconstruction error: %f' % err)
            loss.append(err)
        
        return loss
    
    #Create expected output for our DBN
    def rbm_output(self, X):
        out = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        return out
    
    def rbm_reconstruct(self, X):
        h = tf.nn.sigmoid(tf.matmul(X, self.w) + self.hb)
        reconstruct = tf.nn.sigmoid(tf.matmul(h, tf.transpose(self.w)) + self.vb)
        return reconstruct

## Train tensorflow model 
### _(We'll use the same hyperparameters as previous model)_

In [19]:
rbm_tensorflow = RBM_tensorflow(nv, nh, 1.0, batch_size)
rbm_tensorflow.train(training_set)

Epoch: 0 reconstruction error: 0.995984
Epoch: 1 reconstruction error: 0.995984
Epoch: 2 reconstruction error: 0.995984
Epoch: 3 reconstruction error: 0.995984
Epoch: 4 reconstruction error: 0.995984
Epoch: 5 reconstruction error: 0.995984
Epoch: 6 reconstruction error: 0.995984
Epoch: 7 reconstruction error: 0.995984
Epoch: 8 reconstruction error: 0.995984
Epoch: 9 reconstruction error: 0.995984


[<tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99598414>]